# Heuristics used to assign the defi_flag

1. Assumed entities with '@' or being None are considered as unknown in category and non-defi
2. Used Fuzzy String Matching as WRatio to find category for each entity
- WRatio is used as it combines multiple fuzz metrics (simple ratio, partial ratio, token-sort/token-set) into one score, so it reliably matches names despite extra words, different word order, or punctuation—giving high recall with a single call instead of chaining several methods.
3. Entities that weren't found using Fuzzy String Matching was manually searched with sources including 
- DefiLlama (due to poor performance in WRatio)
- tags/labels in various 'scan' websites 
- their own websites if above two sources did not provide any information.

# Save DefiLlama protocals

In [136]:
import requests, json
baseUrl = 'https://api.llama.fi'
protocols = requests.get(baseUrl + '/protocols').json()

In [137]:
import pandas as pd
protocalsData = pd.DataFrame.from_dict(protocols)
protocalsData.info(); protocalsData.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6027 entries, 0 to 6026
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        6027 non-null   object 
 1   name                      6027 non-null   object 
 2   address                   3252 non-null   object 
 3   symbol                    6027 non-null   object 
 4   url                       6027 non-null   object 
 5   description               6026 non-null   object 
 6   chain                     5613 non-null   object 
 7   logo                      6027 non-null   object 
 8   audits                    6015 non-null   object 
 9   audit_note                6 non-null      object 
 10  gecko_id                  2012 non-null   object 
 11  cmcId                     1580 non-null   object 
 12  category                  6027 non-null   object 
 13  chains                    6027 non-null   object 
 14  module  

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,misrepresentedTokens,forkedFromIds,oracles,tags,stablecoins,language,deadUrl,deadFrom,rugged,deprecated
0,2269,Binance CEX,None,-,https://www.binance.com,Binance is a cryptocurrency exchange which is ...,Multi-Chain,https://icons.llama.fi/binance-cex.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1599,AAVE V3,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,https://aave.com,"Earn interest, borrow assets, and build applic...",Multi-Chain,https://icons.llama.fi/aave-v3.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2272,OKX,None,-,https://www.okx.com,"OKX, formerly known as OKEx, is a Seychelles-b...",Multi-Chain,https://icons.llama.fi/okx.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2275,Bitfinex,None,-,https://www.bitfinex.com,Bitfinex facilitates a graphical trading exper...,Multi-Chain,https://icons.llama.fi/bitfinex.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,182,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,Liquid staking for Ethereum and Polygon. Daily...,Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
protocalsData = protocalsData[['name','address','url','slug','category']]
protocalsData.to_excel("../data/processed/DefiLlama_Protocal.xlsx", index=False)

# Load Wintermute Data

In [139]:
import pandas as pd
DF_PATH = "../data/processed/wintermute_transfers_clean.parquet" 
df = pd.read_parquet(DF_PATH)
display(df.info())
display(print(f"df shape: {df.shape}"))
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429648 entries, 0 to 429647
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   timestamp     429648 non-null  object 
 1   from_address  429648 non-null  object 
 2   from_entity   396923 non-null  object 
 3   to_address    429648 non-null  object 
 4   to_entity     400231 non-null  object 
 5   token         429648 non-null  object 
 6   value         429648 non-null  float64
 7   usd           429648 non-null  float64
 8   tx_hash       429648 non-null  object 
 9   chain         429648 non-null  object 
 10  block_number  429648 non-null  float64
dtypes: float64(3), object(8)
memory usage: 36.1+ MB


None

df shape: (429648, 11)


None

,timestamp,from_address,from_entity,to_address,to_entity,token,value,usd,tx_hash,chain,block_number
0,2025-04-08 12:49:57+00:00,0xb1026b8e7276e7ac75410f1fcbbe21796e8f7526,Camelot,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,USDC,438.108563,438.108563,0x98058c529466064c355bdfea3cfebe399344678f20bb...,arbitrum_one,324218955.0
1,2025-04-08 12:49:54+00:00,0xcda53b1f66614552f834ceef361a8d12a0b8dad8,Uniswap,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,ARB,3837.138536,1062.879700,0x405b146ac6a52b93e5a77760a4b49a87015b5c5e0659...,arbitrum_one,324218945.0
2,2025-04-08 12:49:51+00:00,0xb2cc224c1c9fee385f8ad6a55b4d94e92359dc59,Aerodrome Finance,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,WETH,7.345922,11533.612401,0x7f370fc61bed7241d846c4907a4e616e69ce60ff0192...,base,28663622.0
3,2025-04-08 12:49:51+00:00,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,0x72ab388e2e2f6facef59e3c3fa2c4e29011c2d38,PancakeSwap,USDC,2143.712587,2143.712587,0x88b4fb948e95cc2b9a22c48efe3d6a6788209302e72f...,base,28663622.0
4,2025-04-08 12:49:50+00:00,0x641c00a822e8b671738d32a431a4fb6074e5c79d,Uniswap,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,WETH,0.470011,737.550466,0x39fc603c882c2f5af51b7f8cfc8eb58753f5e42eff94...,arbitrum_one,324218918.0


# Find Unique addresses and entities that Wintermute interacted with

In [140]:
df = df[ df["from_entity"].str.contains("Wintermute", na=False) | df["to_entity"].str.contains("Wintermute", na=False)]
df.shape

(409224, 11)

In [141]:
df_from = df[df['from_entity']!='Wintermute']
df_from = df_from.rename(columns={"from_address": "address", "from_entity": "entity"})

df_to = df[df['to_entity']!='Wintermute']
df_to = df_to.rename(columns={"to_address": "address", "to_entity": "entity"})

df_non_wintermute = (
    pd.concat([df_from, df_to], ignore_index=True)
      .drop_duplicates(subset=["address"])      # keep the first entity you see
      .drop(columns=['from_address','from_entity',
                    'to_address','to_entity'])
      .reset_index(drop=True)
      .sort_values('entity')
)

addresses = df_non_wintermute['address'].dropna().astype(str).unique().tolist()
entities = df_non_wintermute['entity'].dropna().astype(str).unique().tolist()

with open("../outputs/addresses.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(addresses))
with open("../outputs/entities.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(entities))

In [142]:
df_non_wintermute

,timestamp,address,entity,token,value,usd,tx_hash,chain,block_number
407,2025-04-08 12:38:35+00:00,0x0d0e364aa7852291883c162b22d6d81f6355428f,0x,APE,430.000000,173.525640,0x8f606d50154cc5d5edb7c440ed119214186665fb430c...,ethereum,22224235.0
656,2025-04-08 12:19:23+00:00,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x,WETH,0.036676,58.201541,0xc3e3ecac5ecf5edf79fbc3468deb6d6428339f19571e...,ethereum,22224139.0
665,2025-04-08 12:17:35+00:00,0x9ed181da6b359c3ee23c1d6912a6b4b0c349a165,0x,WETH,13.126384,20830.259441,0xb6d39a54ff8790f10a8ccc76a4e22f40812cd61c02e5...,ethereum,22224130.0
222,2025-04-08 12:44:35+00:00,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0x,FLOKI,497512.437811,25.203980,0x222486e095364ea151610417142df5a77847fece4f5a...,ethereum,22224265.0
3569,2025-04-08 11:36:23+00:00,0xad3b67bca8935cb510c8d18bd45f0b94f54a968f,1inch,LDO,3313.869335,2216.541154,0x43b71fdee2f56e08b83a690571cab9b1604014c261be...,ethereum,22223924.0
...,...,...,...,...,...,...,...,...,...
5021,2025-04-08 01:30:48+00:00,35hrqpjwvm72kb9udaorqvttrh331hptbqpzbylothn8,None,SOL,2.515256,270.415166,54zwbr2h45n77srz2fsamyxwcstkfnw3zrzdrkgp7xu3hn...,solana,332007018.0
5022,2025-04-08 01:30:21+00:00,avevdtmwavww5gkjua7icpvhq37tneax5hkfdp6g8rwf,None,WBTC,0.093502,7377.338571,4r4bcfqfwa9apfgs5c1xjhnfijub8d5qjjxamkbyjsfpfa...,solana,332006948.0
5023,2025-04-08 01:30:18+00:00,9odn2ykye1ea2nxdgarty5khrrzptxuklgcrv8ztakhy,None,USDT,98.234477,98.234477,5pkhdg17tyj7nwoqkrb6ifsfwbvyloto7pj65yzngaax1k...,solana,332006942.0
5024,2025-04-08 01:30:15+00:00,5ekr7r53jgt7fey7hnssz9eazsjz2mm9fowbc3sqvwhs,None,USDC,290.002755,290.002755,3eym5f7rn38pm3e1etmdzxdt9tvcdcvmeferpqztspnfyc...,solana,332006934.0


# Create DataFrame to categorize and classify entities

In [143]:
map_df = df_non_wintermute.copy()

In [144]:
map_df[["slug", "category", "source", "confidence"]] = None
mask_unknown = map_df["entity"].isna() | map_df["entity"].str.contains("@", na=False)
map_df.loc[mask_unknown, "confidence"] = 1.0

# Use Fuzzy String Matching with DefiLlama to categorize entities

In [145]:
from rapidfuzz import process, fuzz

proto_url = "https://api.llama.fi/protocols"                         
proto_df  = pd.json_normalize(requests.get(proto_url, timeout=30).json())
proto_df  = proto_df[["name","slug","category"]]

# # Preparation for Fuzzy String Match
# proto_df['name'] = proto_df['name'].str.lower().str.strip()
# map_df['entity'] = map_df['entity'].str.lower().str.strip()

def match_name(ent, cutoff=90):
    hit = process.extractOne(
        ent, proto_df["name"], scorer=fuzz.WRatio, score_cutoff=cutoff
    )
    print(ent, hit)
    if hit:
        row = proto_df.iloc[hit[2]]
        return row.slug, row.category, 1.0
    return None, None, 0.0

subset = map_df.loc[~mask_unknown, "entity"]
hits = subset.apply(match_name)
hits_df = (pd.DataFrame(hits.to_list(),index=subset.index,columns=['slug','category','confidence']))
for col in ['slug','category','confidence']:
    map_df.loc[~mask_unknown,col] = hits_df[col]

0x ('0x', 100.0, 5258)
0x ('0x', 100.0, 5258)
0x ('0x', 100.0, 5258)
0x ('0x', 100.0, 5258)
1inch ('1inch Network', 90.0, 1083)
1inch ('1inch Network', 90.0, 1083)
1inch ('1inch Network', 90.0, 1083)
1inch ('1inch Network', 90.0, 1083)
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Aerodrome Finance None
Arctic Bastion (1inch Resolver) ('Resolv', 90.0, 152)
Backpack Exchange ('Backpack', 90.0, 362)
Backpack Exchange ('Backpack', 90.0, 362)
Bebop ('Bebop', 100.0, 5572)
Binance ('Binance CEX', 90.0, 0)
Binance ('Binance CEX', 90.0, 0)
Binance ('Binance CEX', 90.0, 0)
Binance ('Binance CEX', 90.0, 0)
Binance ('Binance CEX', 90.0

In [146]:
# 4. Label rows that got a perfect defillama hit
sel = (map_df["confidence"] == 1.0) & (~mask_unknown)
map_df.loc[sel, "source"] = (
    map_df["source"].fillna("defillama")
)
unresolved = map_df["confidence"] < 1

In [147]:
(map_df.loc[unresolved,['entity','chain']]
 .drop_duplicates(['entity'])
)

,entity,chain
323,Aerodrome Finance,base
1872,Bitvavo,ethereum
517,Bullish.com,ethereum
8,CoW Protocol,arbitrum_one
2485,Coinhako,ethereum
327,Curve.fi,ethereum
7,Fluid (Instadapp),arbitrum_one
3004,KuCoin,base
3162,Kyber Network,arbitrum_one
306,LiFi,base


# Manually categorize entities that weren't using Fuzzy String Matching 

## 1. Not found with Fuzzy String Match but were in DefiLlama
| name in raw data | name in DefiLlama | slug | category | index in excel |
| ---------------- | ----------------- | ---- | -------- | ----- |
| Aerodrome Finance | Aerodrome V1 | aerodrome-v1 | Dexs | 188 |
| Curve.fi | Curve DEX | curve-dex | Dexs | 43 |
| Fluid (Instadapp) | Fluid Lending | fluid-lending | Lending | 70 |
| KuCoin | Kucoin | kucoin | CEX | 39 |
| Odos | ODOS | odos | DEX Aggregator | 5571 |
| Rainbow.me | Rainbow | rainbow | Wallets | 5427 |
| Sky (MakerDAO) | Sky RWA | sky-rwa | RWA | 320 |
| Velodrome Finance | Velodrome V3 | veledrome | Dexs | 436 |
| Velodrome Finance | Velodrome V3 | veledrome | Dexs | 436 |

## 2. Not found with Fuzzy String Match and in DefiLlama
| name in raw data | category | website |
| ---------------- | ----------------- | ----------------- | 
| Bitvavo | CEX | [etherscan](https://etherscan.io/address/0xaB782bc7D4a2b306825de5a7730034F8F63ee1bC) |
| Bullish.com | CEX | [etherscan](https://etherscan.io/address/0x367c42a6f261ec54ffbecf5f41c226be12a3dca0) |
| CoW Protocol | Dexs | [arbiscan](https://arbiscan.io/token/0xcb8b5cd20bdcaea9a010ac1f8d835824f5c87a04) |
| Coinhako | CEX | [etherscan](https://etherscan.io/address/0xe66baa0b612003af308d78f066bbdb9a5e00ff6c) |
| LiFi | Bridge | [basescan](https://basescan.org/address/0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae) |
| Kyber Network | DEX Aggregator | [arbiscan](https://www.arbiscan.io/address/0xc7d3ab410d49b664d03fe5b1038852ac852b1b29) |
| Meteora (Prev. Mercurial) | Dexs | [own website](https://docs.meteora.ag/) |
| ParaSwap | DEX Aggregator | [basescan](https://basescan.org/accounts/label/paraswap) |
| rsync-builder | MEV Builder | [etherscan](https://etherscan.io/address/0x1f9090aae28b8a3dceadf281b0f12828e676c326) |

## 3. Resolvers
| name in raw data | category |
| ---------------- | ----------------- |
| Arctic Bastion (1inch Resolver) | Dex-Resolver |
| The T Resolver (1inch Resolver) | Dex-Resolver |
| Rizzsolver (Wintermute) | Dex-Resolver |

In [148]:
# manual_rows = [
#     ("Aerodrome Finance", "aerodrome-v1", "Dexs"),
#     ("Curve.fi",          "curve-dex",    "Dexs"),
#     ("Fluid (Instadapp)", "fluid-lending","Lending"),
#     ("KuCoin",            "kucoin",       "CEX"),
#     ("Odos",              "odos",         "DEX Aggregator"),
#     ("Rainbow.me",        "rainbow",      "Wallets"),
#     ("Sky (MakerDAO)",    "sky-rwa",      "RWA"),
#     ("Velodrome Finance", "veledrome",    "Dexs"),
#     ("Bitvavo", None,    "CEX"),
#     ("Bullish.com", None,    "CEX"),
#     ("CoW Protocol", None,    "Dexs"),
#     ("Coinhako", None,    "CEX"),
#     ("LiFi", None,    "Bridge"),
#     ("Kyber Network", None,    "DEX Aggregator"),
#     ("Meteora (Prev. Mercurial)", None,    "Dexs"),
#     ("ParaSwap", None,    "DEX Aggregator"),
#     ("rsync-builder", None,    "MEV Builder"),
#     ("Arctic Bastion (1inch Resolver)", None,    "Dex-Resolver"),
#     ("The T Resolver (1inch Resolver)", None,    "Dex-Resolver"),
#     ("Rizzolver (Wintermute)", None,    "Dex-Resolver"),   
# ]

# manual = (
#     pd.DataFrame(manual_rows, columns=["entity", "slug", "category"])
#       .assign(confidence=1.0, source="defillama")
#       .set_index("entity")
# )
# manual.index = manual.index.str.lower()          # make index case-insensitive

# map_df = map_df.set_index(map_df["entity"].str.lower())
# map_df.update(manual[["slug", "category", "confidence", "source"]])

# map_df.reset_index(drop=True, inplace=True)

import pandas as pd

# 1) each tuple now has FOUR elements → entity, slug, category, source
manual_rows = [
    ("Aerodrome Finance", "aerodrome-v1", "Dexs", "defillama"),
    ("Curve.fi",          "curve-dex",    "Dexs", "defillama"),
    ("Fluid (Instadapp)", "fluid-lending","Lending", "defillama"),
    ("KuCoin",            "kucoin",       "CEX", "defillama"),
    ("Odos",              "odos",         "DEX Aggregator", "defillama"),
    ("Rainbow.me",        "rainbow",      "Wallets", "defillama"),   # ← fixed tuple len
    ("Sky (MakerDAO)",    "sky-rwa",      "RWA", "defillama"),
    ("Velodrome Finance", "veledrome",    "Dexs", "defillama"),
    ("Bitvavo",           None,           "CEX", "etherscan"),
    ("Bullish.com",       None,           "CEX", "etherscan"),
    ("CoW Protocol",      None,           "Dexs", "arbiscan"),
    ("Coinhako",          None,           "CEX", "etherscan"),
    ("LiFi",              None,           "Bridge", "basescan"),
    ("Kyber Network",     None,           "DEX Aggregator", "arbiscan"),
    ("Meteora (Prev. Mercurial)", None,   "Dexs", "own website"),
    ("ParaSwap",          None,           "DEX Aggregator", "basescan"),
    ("rsync-builder",     None,           "MEV Builder", "etherscan"),
    ("Arctic Bastion (1inch Resolver)", None, "Dex-Resolver", "manual"),
    ("The T Resolver (1inch Resolver)", None, "Dex-Resolver", "manual"),
    ("Rizzolver (Wintermute)", None,      "Dex-Resolver", "manual"),
]

# 2) build the override table WITH a 'source' column
manual = (
    pd.DataFrame(manual_rows,
                 columns=["entity", "slug", "category", "source"])
      .assign(confidence=0.95)                 # or 1.0 if you wish
      .set_index("entity")
)
manual.index = manual.index.str.lower()        # case–insensitive key

# 3) align map_df on the same lowercase entity key
map_df = map_df.set_index(map_df["entity"].str.lower())

# 4) push slug, category, confidence, **source** into map_df
cols_to_update = ["slug", "category", "confidence", "source"]
map_df.update(manual[cols_to_update])

# 5) (optional) restore original index
map_df.reset_index(drop=True, inplace=True)


In [149]:
map_df['category'].value_counts()

category
Dexs                 458
CEX                  121
DEX Aggregator        15
Bridge                15
Liquid Staking         8
Dex-Resolver           4
Wallets                3
RWA                    2
Stablecoin Issuer      1
Basis Trading          1
Lending                1
MEV Builder            1
Name: count, dtype: int64

# Classify category into defi vs non-defi 

In [150]:
DEFI_KEEP = {
    "Dexs",
    "DEX Aggregator",
    "Bridge",
    "Liquid Staking",
    "Lending",
    "Dex-Resolver",
    "MEV Builder",
    "RWA",
}

map_df_defi = map_df[map_df["category"].isin(DEFI_KEEP)]
defi_entities = map_df_defi['entity'].drop_duplicates().values.tolist()

In [151]:
with open("../outputs/defi_entities.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(defi_entities))

In [152]:
defi_true = defi_entities.copy()
defi_true.append("Wintermute")
df_defi = df[df["from_entity"].isin(defi_true) & df["to_entity"].isin(defi_true)].copy()
display(print(f"df_defi shape: {df_defi.shape}"))
display(df_defi.head())

df_defi shape: (366056, 11)


None

,timestamp,from_address,from_entity,to_address,to_entity,token,value,usd,tx_hash,chain,block_number
0,2025-04-08 12:49:57+00:00,0xb1026b8e7276e7ac75410f1fcbbe21796e8f7526,Camelot,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,USDC,438.108563,438.108563,0x98058c529466064c355bdfea3cfebe399344678f20bb...,arbitrum_one,324218955.0
1,2025-04-08 12:49:54+00:00,0xcda53b1f66614552f834ceef361a8d12a0b8dad8,Uniswap,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,ARB,3837.138536,1062.879700,0x405b146ac6a52b93e5a77760a4b49a87015b5c5e0659...,arbitrum_one,324218945.0
2,2025-04-08 12:49:51+00:00,0xb2cc224c1c9fee385f8ad6a55b4d94e92359dc59,Aerodrome Finance,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,WETH,7.345922,11533.612401,0x7f370fc61bed7241d846c4907a4e616e69ce60ff0192...,base,28663622.0
3,2025-04-08 12:49:51+00:00,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,0x72ab388e2e2f6facef59e3c3fa2c4e29011c2d38,PancakeSwap,USDC,2143.712587,2143.712587,0x88b4fb948e95cc2b9a22c48efe3d6a6788209302e72f...,base,28663622.0
4,2025-04-08 12:49:50+00:00,0x641c00a822e8b671738d32a431a4fb6074e5c79d,Uniswap,0x51c72848c68a965f66fa7a88855f9f7784502a7f,Wintermute,WETH,0.470011,737.550466,0x39fc603c882c2f5af51b7f8cfc8eb58753f5e42eff94...,arbitrum_one,324218918.0


# Save defi transaction on chain data 

In [153]:
df_defi.to_parquet("../data/processed/wintermute_transfers_clean_defi.parquet", index=False)

# Save Unique addresses and entities with categories

In [154]:
map_df.to_excel("../data/processed/unique_addresses_entities_categories.xlsx", index=False)

In [155]:
map_df

,timestamp,address,entity,token,value,usd,tx_hash,chain,block_number,slug,category,source,confidence
0,2025-04-08 12:38:35+00:00,0x0d0e364aa7852291883c162b22d6d81f6355428f,0x,APE,430.000000,173.525640,0x8f606d50154cc5d5edb7c440ed119214186665fb430c...,ethereum,22224235.0,0x,DEX Aggregator,defillama,1.0
1,2025-04-08 12:19:23+00:00,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x,WETH,0.036676,58.201541,0xc3e3ecac5ecf5edf79fbc3468deb6d6428339f19571e...,ethereum,22224139.0,0x,DEX Aggregator,defillama,1.0
2,2025-04-08 12:17:35+00:00,0x9ed181da6b359c3ee23c1d6912a6b4b0c349a165,0x,WETH,13.126384,20830.259441,0xb6d39a54ff8790f10a8ccc76a4e22f40812cd61c02e5...,ethereum,22224130.0,0x,DEX Aggregator,defillama,1.0
3,2025-04-08 12:44:35+00:00,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0x,FLOKI,497512.437811,25.203980,0x222486e095364ea151610417142df5a77847fece4f5a...,ethereum,22224265.0,0x,DEX Aggregator,defillama,1.0
4,2025-04-08 11:36:23+00:00,0xad3b67bca8935cb510c8d18bd45f0b94f54a968f,1inch,LDO,3313.869335,2216.541154,0x43b71fdee2f56e08b83a690571cab9b1604014c261be...,ethereum,22223924.0,1inch-network,DEX Aggregator,defillama,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5021,2025-04-08 01:30:48+00:00,35hrqpjwvm72kb9udaorqvttrh331hptbqpzbylothn8,None,SOL,2.515256,270.415166,54zwbr2h45n77srz2fsamyxwcstkfnw3zrzdrkgp7xu3hn...,solana,332007018.0,None,None,None,1.0
5022,2025-04-08 01:30:21+00:00,avevdtmwavww5gkjua7icpvhq37tneax5hkfdp6g8rwf,None,WBTC,0.093502,7377.338571,4r4bcfqfwa9apfgs5c1xjhnfijub8d5qjjxamkbyjsfpfa...,solana,332006948.0,None,None,None,1.0
5023,2025-04-08 01:30:18+00:00,9odn2ykye1ea2nxdgarty5khrrzptxuklgcrv8ztakhy,None,USDT,98.234477,98.234477,5pkhdg17tyj7nwoqkrb6ifsfwbvyloto7pj65yzngaax1k...,solana,332006942.0,None,None,None,1.0
5024,2025-04-08 01:30:15+00:00,5ekr7r53jgt7fey7hnssz9eazsjz2mm9fowbc3sqvwhs,None,USDC,290.002755,290.002755,3eym5f7rn38pm3e1etmdzxdt9tvcdcvmeferpqztspnfyc...,solana,332006934.0,None,None,None,1.0
